In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import io
import uuid
from reportlab.lib.pagesizes import letter, landscape
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors


from reportlab.lib.units import mm
from reportlab.platypus import PageBreak

# ----- FUNCIONES DE ESTILO Y EXPORTACIÓN -----
def wandb_style_table(df):
    styles = """
    <style>
        .wandb-table {
            border-collapse: collapse;
            width: 100%;
            table-layout: fixed;
            font-family: "Segoe UI", Roboto, Helvetica, Arial, sans-serif;
            font-size: 14px;
            border: 1px solid #e0e0e0;
            border-radius: 6px;
            overflow: hidden;
            box-shadow: 0 2px 6px rgba(0, 0, 0, 0.05);
        }
        .wandb-table th, .wandb-table td {
            padding: 10px 8px;
            border: 1px solid #ddd;
            text-align: center;
            overflow: hidden;
            text-overflow: ellipsis;
            white-space: nowrap;
        }
        .wandb-table th {
            background-color: #f0f7ff;
            color: #333;
            font-weight: 600;
            text-align: center;
        }
        .wandb-table tr:nth-child(even) {
            background-color: #f9fcff;
        }
        .wandb-table tr:hover {
            background-color: #e6f2ff;
        }
    </style>
    """
    return HTML(styles + df.to_html(index=False, classes="wandb-table", escape=False))



def download_svg(df, filename="wandb_table.svg"):
    html = wandb_style_table(df).data
    height = len(df) * 35 + 60
    width = min(1600, len(df.columns) * 130 + 50)
    svg_template = f'''
    <svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}">
        <foreignObject width="100%" height="100%">
            <div xmlns="http://www.w3.org/1999/xhtml">
                {html}
            </div>
        </foreignObject>
    </svg>
    '''
    with open(filename, "w") as f:
        f.write(svg_template)
    display(HTML(f'<a href="{filename}" download>Descargar SVG</a>'))


def export_pdf_from_dataframe(df, filename="table.pdf", wide_first_col=False):

    # Convertir todos los valores a string y truncar decimales si es necesario
    def format_value(val):
        try:
            f = float(val)
            if f.is_integer():
                return str(int(f))
            return f"{f:.10f}".rstrip("0").rstrip(".")
        except:
            return str(val)

    formatted_df = df.applymap(format_value)

    # Crear tabla de datos
    data = [list(formatted_df.columns)] + formatted_df.values.tolist()

    # Tamaño dinámico en función de número de columnas y filas
    col_width = 30 * mm
    if wide_first_col:
        col_widths = [80 * mm] + [col_width] * (len(df.columns) - 1)
        total_width = 80 * mm + col_width * (len(df.columns) - 1)
    else:
        col_widths = [col_width] * len(df.columns)
        total_width = col_width * len(df.columns)
    
    row_height = 20  # Altura estimada para evitar separación
    total_height = row_height * len(data)

    # Documento PDF con tamaño justo a la tabla
    pdf = SimpleDocTemplate(
        filename,
        pagesize=(total_width, total_height),
        leftMargin=0,
        rightMargin=0,
        topMargin=0,
        bottomMargin=0
    )

    table = Table(data, colWidths=col_widths, repeatRows=1)

    # Estilo moderno estilo wandb
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#f0f7ff')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor('#333333')),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f9fcff')]),
        ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.HexColor('#cccccc')),
        ('BOX', (0, 0), (-1, -1), 0.5, colors.HexColor('#cccccc')),
    ]))

    pdf.build([table])

    display(HTML(f'<a href="{filename}" download>Descargar PDF</a>'))


# ----- INTERFAZ Y FUNCIONALIDAD -----
upload = widgets.FileUpload(accept=".csv", multiple=False)
output = widgets.Output()

def on_upload_change(change):
    with output:
        clear_output()

        if not upload.value:
            return

        try:
            if isinstance(upload.value, tuple) and len(upload.value) > 0:
                uploaded_file = upload.value[0]
                file_content = uploaded_file['content']
                df = pd.read_csv(io.BytesIO(file_content))
            else:
                display(HTML("<div style='color:red'>Error al leer el archivo</div>"))
                return

            columns = widgets.SelectMultiple(
                options=df.columns,
                description="Columnas",
                layout={'width': '400px'}
            )

            filter_col = widgets.Dropdown(
                options=df.columns,
                description="Filtrar por"
            )

            filter_val = widgets.Text(
                description="Valor",
                placeholder="Texto o número"
            )

            sort_col = widgets.Dropdown(
                options=df.columns,
                description="Ordenar por"
            )

            sort_order = widgets.ToggleButtons(
                options=[('Ascendente', True), ('Descendente', False)],
                description='Orden:'
            )

            row_count = widgets.BoundedIntText(
                value=10,
                min=1,
                max=len(df),
                description="Filas"
            )

            update_btn = widgets.Button(
                description="Actualizar Tabla",
                button_style='success'
            )

            export_svg_btn = widgets.Button(
                description="Exportar SVG",
                button_style='info'
            )

            export_pdf_btn = widgets.Button(
                description="Exportar PDF",
                button_style='info'
            )
            
            wide_first_col = widgets.Checkbox(
                value=False,
                description="Primera columna más ancha",
                indent=False
            )


            table_output = widgets.Output()
            
            def get_filtered_df():
                filtered_df = df.copy()

                if filter_col.value and filter_val.value:
                    try:
                        if pd.api.types.is_numeric_dtype(df[filter_col.value]):
                            filtered_df = filtered_df[filtered_df[filter_col.value] == float(filter_val.value)]
                        else:
                            filtered_df = filtered_df[
                                filtered_df[filter_col.value].astype(str).str.contains(
                                    filter_val.value, case=False)
                            ]
                    except ValueError:
                        return None

                if columns.value:
                    filtered_df = filtered_df[list(columns.value)]

                if sort_col.value:
                    filtered_df = filtered_df.sort_values(by=sort_col.value, ascending=sort_order.value)

                return filtered_df.head(row_count.value)


            def update_table(b):
                with table_output:
                    table_output.clear_output()
                    result_df = get_filtered_df()
                    if result_df is None:
                        display(HTML("<div style='color:red'>Error: Valor no válido para esta columna</div>"))
                    else:
                        display(wandb_style_table(result_df))

            def on_export_svg(b):
                result_df = get_filtered_df()
                if result_df is not None:
                    filename = f"tabla_{uuid.uuid4().hex}.svg"
                    download_svg(result_df, filename)

            def on_export_pdf(b):
                result_df = get_filtered_df()
                if result_df is not None:
                    export_pdf_from_dataframe(result_df, wide_first_col=wide_first_col.value)


            update_btn.on_click(update_table)
            export_svg_btn.on_click(on_export_svg)
            export_pdf_btn.on_click(on_export_pdf)

            controls = widgets.VBox([
                widgets.HBox([columns]),
                widgets.HBox([filter_col, filter_val]),
                widgets.HBox([sort_col, sort_order]),
                widgets.HBox([row_count]),
                wide_first_col,
                widgets.HBox([update_btn, export_svg_btn, export_pdf_btn]),
                table_output
            ])


            display(controls)
            update_table(None)

        except Exception as e:
            display(HTML(f"<div style='color:red'>Error: {str(e)}</div>"))

upload.observe(on_upload_change, names='value')
display(upload, output)


FileUpload(value=(), accept='.csv', description='Upload')

Output()